In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST", one_hot = True)

Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/train-images-idx3-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/train-labels-idx1-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/t10k-images-idx3-ubyte.gz
Extracting /Users/neelbakshi/Documents/Machine Learning/Datasets/MNIST/t10k-labels-idx1-ubyte.gz


In [10]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network(x):
    hidden_layer_1_nodes = 500
    hidden_layer_2_nodes = 300
    hidden_layer_3_nodes = 200
    hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([784, hidden_layer_1_nodes])), 
                      'biases': tf.Variable(tf.random_normal([hidden_layer_1_nodes]))}
    
    hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([hidden_layer_1_nodes, hidden_layer_2_nodes])),
                     'biases': tf.Variable(tf.random_normal([hidden_layer_2_nodes]))}
    
    hidden_layer_3 = {'weights': tf.Variable(tf.random_normal([hidden_layer_2_nodes, hidden_layer_3_nodes])),
                     'biases': tf.Variable(tf.random_normal([hidden_layer_3_nodes]))}
    
    output_layer = {'weights': tf.Variable(tf.random_normal([hidden_layer_3_nodes, 10])),
                   'biases': tf.Variable(tf.random_normal([10]))}
    
    l1 = tf.add(tf.matmul(x, hidden_layer_1['weights']), hidden_layer_1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_layer_2['weights']), hidden_layer_2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_layer_3['weights']), hidden_layer_3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
    
    return output
    

In [26]:
def train_network(x):
    predicted_output = neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=predicted_output))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in xrange(epochs):
            for _ in range(int(mnist.train.num_examples/200)):
                epoch_loss = 0
                epoch_x, epoch_y = mnist.train.next_batch(200)
                c, _ = sess.run([cost, optimizer], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                
                print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)
            
            correct = tf.equal(tf.argmax(predicted_output, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [ ]:
train_network(x)

('Epoch', 0, 'completed out of', 10, 'loss:', 48036.57421875)
('Epoch', 0, 'completed out of', 10, 'loss:', 39979.6953125)
('Epoch', 0, 'completed out of', 10, 'loss:', 39170.34375)
('Epoch', 0, 'completed out of', 10, 'loss:', 32512.3125)
('Epoch', 0, 'completed out of', 10, 'loss:', 28432.859375)
('Epoch', 0, 'completed out of', 10, 'loss:', 22437.89453125)
('Epoch', 0, 'completed out of', 10, 'loss:', 22792.765625)
('Epoch', 0, 'completed out of', 10, 'loss:', 24341.875)
('Epoch', 0, 'completed out of', 10, 'loss:', 21034.28515625)
('Epoch', 0, 'completed out of', 10, 'loss:', 20803.595703125)
('Epoch', 0, 'completed out of', 10, 'loss:', 18502.439453125)
('Epoch', 0, 'completed out of', 10, 'loss:', 17169.5078125)
('Epoch', 0, 'completed out of', 10, 'loss:', 19188.216796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 16081.125)
('Epoch', 0, 'completed out of', 10, 'loss:', 17370.470703125)
('Epoch', 0, 'completed out of', 10, 'loss:', 13829.0498046875)
('Epoch', 0, 'completed o

('Epoch', 0, 'completed out of', 10, 'loss:', 1731.6650390625)
('Epoch', 0, 'completed out of', 10, 'loss:', 1079.184326171875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1351.1134033203125)
('Epoch', 0, 'completed out of', 10, 'loss:', 1722.1580810546875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1607.694091796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1443.717529296875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1367.5858154296875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1267.909423828125)
('Epoch', 0, 'completed out of', 10, 'loss:', 864.46466064453125)
('Epoch', 0, 'completed out of', 10, 'loss:', 1561.8125)
('Epoch', 0, 'completed out of', 10, 'loss:', 1282.31591796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1711.81591796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1676.162841796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1577.2010498046875)
('Epoch', 0, 'completed out of', 10, 'loss:', 1290.4798583984375)
('Epoch', 0, 'completed out of'

('Epoch', 0, 'completed out of', 10, 'loss:', 554.64111328125)
('Epoch', 0, 'completed out of', 10, 'loss:', 820.0101318359375)
('Epoch', 0, 'completed out of', 10, 'loss:', 898.90966796875)
('Epoch', 0, 'completed out of', 10, 'loss:', 491.667724609375)
('Epoch', 0, 'completed out of', 10, 'loss:', 891.828369140625)
('Epoch', 0, 'completed out of', 10, 'loss:', 832.25408935546875)
('Epoch', 0, 'completed out of', 10, 'loss:', 693.85040283203125)
('Epoch', 0, 'completed out of', 10, 'loss:', 992.01190185546875)
('Epoch', 0, 'completed out of', 10, 'loss:', 692.927490234375)
('Epoch', 0, 'completed out of', 10, 'loss:', 788.20892333984375)
('Epoch', 0, 'completed out of', 10, 'loss:', 1223.7607421875)
('Epoch', 0, 'completed out of', 10, 'loss:', 525.85675048828125)
('Epoch', 0, 'completed out of', 10, 'loss:', 717.52508544921875)
('Epoch', 0, 'completed out of', 10, 'loss:', 551.9664306640625)
('Epoch', 0, 'completed out of', 10, 'loss:', 525.98028564453125)
('Epoch', 1, 'completed out

('Epoch', 1, 'completed out of', 10, 'loss:', 529.97271728515625)
('Epoch', 1, 'completed out of', 10, 'loss:', 593.683837890625)
('Epoch', 1, 'completed out of', 10, 'loss:', 401.72488403320312)
('Epoch', 1, 'completed out of', 10, 'loss:', 751.04974365234375)
('Epoch', 1, 'completed out of', 10, 'loss:', 502.54196166992188)
('Epoch', 1, 'completed out of', 10, 'loss:', 259.7188720703125)
('Epoch', 1, 'completed out of', 10, 'loss:', 667.128662109375)
('Epoch', 1, 'completed out of', 10, 'loss:', 506.40859985351562)
('Epoch', 1, 'completed out of', 10, 'loss:', 626.07623291015625)
('Epoch', 1, 'completed out of', 10, 'loss:', 825.127685546875)
('Epoch', 1, 'completed out of', 10, 'loss:', 451.06475830078125)
('Epoch', 1, 'completed out of', 10, 'loss:', 771.925048828125)
('Epoch', 1, 'completed out of', 10, 'loss:', 774.55657958984375)
('Epoch', 1, 'completed out of', 10, 'loss:', 873.82080078125)
('Epoch', 1, 'completed out of', 10, 'loss:', 413.86639404296875)
('Epoch', 1, 'complete

('Epoch', 1, 'completed out of', 10, 'loss:', 345.15164184570312)
('Epoch', 1, 'completed out of', 10, 'loss:', 445.1357421875)
('Epoch', 1, 'completed out of', 10, 'loss:', 402.36618041992188)
('Epoch', 1, 'completed out of', 10, 'loss:', 790.50616455078125)
('Epoch', 1, 'completed out of', 10, 'loss:', 238.06440734863281)
('Epoch', 1, 'completed out of', 10, 'loss:', 550.44146728515625)
('Epoch', 1, 'completed out of', 10, 'loss:', 678.5496826171875)
('Epoch', 1, 'completed out of', 10, 'loss:', 518.65264892578125)
('Epoch', 1, 'completed out of', 10, 'loss:', 724.93756103515625)
('Epoch', 1, 'completed out of', 10, 'loss:', 459.27264404296875)
('Epoch', 1, 'completed out of', 10, 'loss:', 692.42218017578125)
('Epoch', 1, 'completed out of', 10, 'loss:', 467.8111572265625)
('Epoch', 1, 'completed out of', 10, 'loss:', 384.705322265625)
('Epoch', 1, 'completed out of', 10, 'loss:', 584.38629150390625)
('Epoch', 1, 'completed out of', 10, 'loss:', 693.1278076171875)
('Epoch', 1, 'compl

...

('Epoch', 7, 'completed out of', 10, 'loss:', 94.041885375976562)
('Epoch', 7, 'completed out of', 10, 'loss:', 48.263389587402344)
('Epoch', 7, 'completed out of', 10, 'loss:', 84.909393310546875)
('Epoch', 7, 'completed out of', 10, 'loss:', 77.102973937988281)
('Epoch', 7, 'completed out of', 10, 'loss:', 101.67825317382812)
('Epoch', 7, 'completed out of', 10, 'loss:', 72.125762939453125)
('Epoch', 7, 'completed out of', 10, 'loss:', 58.644275665283203)
('Epoch', 7, 'completed out of', 10, 'loss:', 51.609535217285156)
('Epoch', 7, 'completed out of', 10, 'loss:', 45.174049377441406)
('Epoch', 7, 'completed out of', 10, 'loss:', 9.3789787292480469)
('Epoch', 7, 'completed out of', 10, 'loss:', 50.128250122070312)
('Epoch', 7, 'completed out of', 10, 'loss:', 99.222076416015625)
('Epoch', 7, 'completed out of', 10, 'loss:', 34.110084533691406)
('Epoch', 7, 'completed out of', 10, 'loss:', 38.013984680175781)
('Epoch', 7, 'completed out of', 10, 'loss:', 81.943855285644531)
('Epoch', 

('Epoch', 8, 'completed out of', 10, 'loss:', 97.53369140625)
('Epoch', 8, 'completed out of', 10, 'loss:', 67.184928894042969)
('Epoch', 8, 'completed out of', 10, 'loss:', 29.106479644775391)
('Epoch', 8, 'completed out of', 10, 'loss:', 41.469902038574219)
('Epoch', 8, 'completed out of', 10, 'loss:', 81.992599487304688)
('Epoch', 8, 'completed out of', 10, 'loss:', 18.884075164794922)
('Epoch', 8, 'completed out of', 10, 'loss:', 22.892578125)
('Epoch', 8, 'completed out of', 10, 'loss:', 4.1597371101379395)
('Epoch', 8, 'completed out of', 10, 'loss:', 57.383159637451172)
('Epoch', 8, 'completed out of', 10, 'loss:', 40.944698333740234)
('Epoch', 8, 'completed out of', 10, 'loss:', 8.9127597808837891)
('Epoch', 8, 'completed out of', 10, 'loss:', 86.435935974121094)
('Epoch', 8, 'completed out of', 10, 'loss:', 50.339176177978516)
('Epoch', 8, 'completed out of', 10, 'loss:', 6.8291821479797363)
('Epoch', 8, 'completed out of', 10, 'loss:', 42.423004150390625)
('Epoch', 8, 'comple

('Epoch', 8, 'completed out of', 10, 'loss:', 41.511943817138672)
('Epoch', 8, 'completed out of', 10, 'loss:', 44.562831878662109)
('Epoch', 8, 'completed out of', 10, 'loss:', 59.667499542236328)
('Epoch', 8, 'completed out of', 10, 'loss:', 112.88442230224609)
('Epoch', 8, 'completed out of', 10, 'loss:', 29.677072525024414)
('Epoch', 8, 'completed out of', 10, 'loss:', 129.62187194824219)
('Epoch', 8, 'completed out of', 10, 'loss:', 93.03607177734375)
('Epoch', 8, 'completed out of', 10, 'loss:', 24.494552612304688)
('Epoch', 8, 'completed out of', 10, 'loss:', 6.2753081321716309)
('Epoch', 8, 'completed out of', 10, 'loss:', 85.731536865234375)
('Epoch', 8, 'completed out of', 10, 'loss:', 85.009872436523438)
('Epoch', 8, 'completed out of', 10, 'loss:', 21.511846542358398)
('Epoch', 8, 'completed out of', 10, 'loss:', 95.357521057128906)
('Epoch', 8, 'completed out of', 10, 'loss:', 103.56336212158203)
('Epoch', 8, 'completed out of', 10, 'loss:', 89.05224609375)
('Epoch', 8, 'c

('Epoch', 9, 'completed out of', 10, 'loss:', 32.988563537597656)
('Epoch', 9, 'completed out of', 10, 'loss:', 46.290733337402344)
('Epoch', 9, 'completed out of', 10, 'loss:', 34.201278686523438)
('Epoch', 9, 'completed out of', 10, 'loss:', 70.524147033691406)
('Epoch', 9, 'completed out of', 10, 'loss:', 71.229835510253906)
('Epoch', 9, 'completed out of', 10, 'loss:', 55.454391479492188)
('Epoch', 9, 'completed out of', 10, 'loss:', 49.178817749023438)
('Epoch', 9, 'completed out of', 10, 'loss:', 6.9473390579223633)
('Epoch', 9, 'completed out of', 10, 'loss:', 82.098213195800781)
('Epoch', 9, 'completed out of', 10, 'loss:', 44.564178466796875)
('Epoch', 9, 'completed out of', 10, 'loss:', 23.948236465454102)
('Epoch', 9, 'completed out of', 10, 'loss:', 12.412954330444336)
('Epoch', 9, 'completed out of', 10, 'loss:', 21.300651550292969)
('Epoch', 9, 'completed out of', 10, 'loss:', 72.65826416015625)
('Epoch', 9, 'completed out of', 10, 'loss:', 31.987026214599609)
('Epoch', 9

('Epoch', 9, 'completed out of', 10, 'loss:', 36.799087524414062)
('Epoch', 9, 'completed out of', 10, 'loss:', 64.075851440429688)
('Epoch', 9, 'completed out of', 10, 'loss:', 57.365604400634766)
('Epoch', 9, 'completed out of', 10, 'loss:', 21.673263549804688)
('Epoch', 9, 'completed out of', 10, 'loss:', 23.136663436889648)
('Epoch', 9, 'completed out of', 10, 'loss:', 15.731208801269531)
